# Microsoft Fabric API - Complete API Exploration

This notebook systematically explores the **ENTIRE** `microsoft-fabric-api` library structure to document all available methods across all modules.

**What this notebook documents:**
- All top-level modules in `fabric_client` (core, admin, workload-specific modules, etc.)
- All sub-modules within each top-level module
- All methods with signatures and docstrings
- All properties and attributes
- Complete API surface area

**Output:** `fabric-api-complete-documentation.json` with comprehensive API documentation

## Setup and Authentication

In [ ]:
%pip install microsoft-fabric-api azure-identity

In [ ]:
import os
import inspect
import json
from azure.identity import ClientSecretCredential
from microsoft_fabric_api import FabricClient

In [ ]:
# Configure credentials
client_id = os.getenv("AZURE_CLIENT_ID", "your-client-id-here")
client_secret = os.getenv("AZURE_CLIENT_SECRET", "your-client-secret-here")
tenant_id = os.getenv("AZURE_TENANT_ID", "your-tenant-id-here")

credential = ClientSecretCredential(
    tenant_id=tenant_id,
    client_id=client_id,
    client_secret=client_secret
)

fabric_client = FabricClient(token_credential=credential)
print("✓ Client initialized")

## Complete API Structure Extraction

In [ ]:
def get_method_details(obj, method_name):
    """Extract detailed information about a method."""
    method = getattr(obj, method_name)
    details = {
        'name': method_name,
        'signature': str(inspect.signature(method)) if callable(method) else None,
        'docstring': inspect.getdoc(method),
        'type': type(method).__name__
    }
    return details

def explore_object(obj, path='', max_depth=5, current_depth=0):
    """Recursively explore an object's structure."""
    if current_depth >= max_depth:
        return {}
    
    structure = {
        'path': path,
        'type': type(obj).__name__,
        'attributes': {},
        'methods': {}
    }
    
    for attr_name in dir(obj):
        if attr_name.startswith('_'):
            continue
        
        try:
            attr = getattr(obj, attr_name)
            new_path = f"{path}.{attr_name}" if path else attr_name
            
            if callable(attr):
                structure['methods'][attr_name] = get_method_details(obj, attr_name)
            else:
                # Non-callable attribute - might be a sub-module
                structure['attributes'][attr_name] = {
                    'type': type(attr).__name__,
                    'path': new_path
                }
        except Exception as e:
            structure['attributes'][attr_name] = {'error': str(e)}
    
    return structure

In [ ]:
# Explore FabricClient top-level
print("Exploring FabricClient...")
print("=" * 80)

client_structure = explore_object(fabric_client, 'fabric_client')

print("\nTop-level attributes:")
for attr_name, attr_info in client_structure['attributes'].items():
    print(f"  • {attr_name}: {attr_info['type']}")

print("\nTop-level methods:")
for method_name in client_structure['methods'].keys():
    print(f"  • {method_name}()")

## Discover All Top-Level Modules

Explore all modules in fabric_client, not just core.

In [ ]:
# Discover all top-level modules (not just core)
print("Discovering all top-level modules in fabric_client...")
print("=" * 80)

all_top_level_modules = {}

for attr_name in dir(fabric_client):
    if not attr_name.startswith('_'):
        try:
            attr = getattr(fabric_client, attr_name)
            if not callable(attr):
                all_top_level_modules[attr_name] = {
                    'object': attr,
                    'type': type(attr).__name__,
                    'path': f'fabric_client.{attr_name}'
                }
                print(f"  • {attr_name}: {type(attr).__name__}")
        except Exception as e:
            print(f"  ✗ {attr_name}: Error - {str(e)}")

print(f"\nFound {len(all_top_level_modules)} top-level modules")

In [ ]:
# Explore sub-modules within each top-level module
all_modules_to_explore = {}

for top_name, top_info in all_top_level_modules.items():
    top_obj = top_info['object']
    
    print(f"\nExploring {top_name}...")
    print("-" * 60)
    
    # Check for sub-modules
    for attr_name in dir(top_obj):
        if not attr_name.startswith('_'):
            try:
                attr = getattr(top_obj, attr_name)
                if not callable(attr):
                    full_path = f"{top_name}.{attr_name}"
                    all_modules_to_explore[full_path] = {
                        'object': attr,
                        'type': type(attr).__name__,
                        'path': f'fabric_client.{full_path}'
                    }
                    print(f"  • {full_path}: {type(attr).__name__}")
            except:
                pass

print(f"\n{'='*80}")
print(f"Total modules to explore in detail: {len(all_modules_to_explore)}")

## Explore Each Module in Detail

Document all methods across all modules in the API.

In [ ]:
# Detailed exploration of ALL modules (not just core)
api_documentation = {
    'client': 'FabricClient',
    'version': '0.1.0b2',
    'top_level_modules': list(all_top_level_modules.keys()),
    'client_methods': [],
    'all_modules': {}
}

# Document top-level client methods
print("Exploring FabricClient methods...")
print("=" * 80)

for attr_name in dir(fabric_client):
    if not attr_name.startswith('_'):
        try:
            attr = getattr(fabric_client, attr_name)
            if callable(attr):
                method_info = {
                    'name': attr_name,
                    'signature': str(inspect.signature(attr)),
                    'docstring': inspect.getdoc(attr) or 'No documentation available'
                }
                api_documentation['client_methods'].append(method_info)
                print(f"  • {attr_name}{method_info['signature']}")
        except Exception as e:
            print(f"  ✗ {attr_name}: {str(e)}")

print(f"\nFound {len(api_documentation['client_methods'])} client methods")

# Document all discovered modules
for module_path, module_meta in all_modules_to_explore.items():
    module_obj = module_meta['object']
    
    print(f"\n{'='*80}")
    print(f"Module: fabric_client.{module_path}")
    print(f"{'='*80}")
    
    module_info = {
        'path': f'fabric_client.{module_path}',
        'type': module_meta['type'],
        'methods': [],
        'properties': []
    }
    
    # Get all methods and properties
    for attr_name in dir(module_obj):
        if not attr_name.startswith('_'):
            try:
                attr = getattr(module_obj, attr_name)
                if callable(attr):
                    method_info = {
                        'name': attr_name,
                        'signature': str(inspect.signature(attr)),
                        'docstring': inspect.getdoc(attr) or 'No documentation available'
                    }
                    module_info['methods'].append(method_info)
                    
                    print(f"\n  Method: {attr_name}")
                    print(f"  Signature: {method_info['signature']}")
                    if method_info['docstring'] and method_info['docstring'] != 'No documentation available':
                        doc_preview = method_info['docstring'][:200] + "..." if len(method_info['docstring']) > 200 else method_info['docstring']
                        print(f"  Documentation:\n    {doc_preview}")
                else:
                    # Non-callable property
                    property_info = {
                        'name': attr_name,
                        'type': type(attr).__name__
                    }
                    module_info['properties'].append(property_info)
                    
            except Exception as e:
                print(f"  ✗ Error exploring {attr_name}: {str(e)}")
    
    api_documentation['all_modules'][module_path] = module_info

print(f"\n{'='*80}")
print(f"Exploration complete!")
print(f"  • Top-level modules: {len(api_documentation['top_level_modules'])}")
print(f"  • Client methods: {len(api_documentation['client_methods'])}")
print(f"  • Total sub-modules explored: {len(api_documentation['all_modules'])}")
print(f"  • Total methods found: {sum(len(m['methods']) for m in api_documentation['all_modules'].values())}")

## Export API Documentation to JSON

In [ ]:
# Save to JSON file
output_file = 'fabric-api-complete-documentation.json'

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(api_documentation, f, indent=2, ensure_ascii=False)

print(f"✓ Complete API documentation saved to {output_file}")
print(f"\nSummary:")
print(f"  • Top-level modules: {len(api_documentation['top_level_modules'])}")
print(f"  • Client methods: {len(api_documentation['client_methods'])}")
print(f"  • Total sub-modules: {len(api_documentation['all_modules'])}")

total_methods = sum(len(m['methods']) for m in api_documentation['all_modules'].values())
print(f"  • Total methods: {total_methods + len(api_documentation['client_methods'])}")

print(f"\nBy module:")
for module_path, module_info in sorted(api_documentation['all_modules'].items()):
    print(f"  • {module_path}: {len(module_info['methods'])} methods, {len(module_info['properties'])} properties")

## Summary Statistics

In [ ]:
# Create a detailed summary
print("Microsoft Fabric API - Complete Summary")
print("=" * 80)

print(f"\nTop-Level Modules in fabric_client:")
for module_name in sorted(api_documentation['top_level_modules']):
    print(f"  • {module_name}")

print(f"\n\nFabricClient Direct Methods: {len(api_documentation['client_methods'])}")
if api_documentation['client_methods']:
    for method in api_documentation['client_methods']:
        print(f"  • {method['name']}{method['signature']}")

total_methods = sum(len(m['methods']) for m in api_documentation['all_modules'].values())
total_properties = sum(len(m.get('properties', [])) for m in api_documentation['all_modules'].values())

print(f"\n\nTotal Statistics:")
print(f"  • Top-level modules: {len(api_documentation['top_level_modules'])}")
print(f"  • Sub-modules explored: {len(api_documentation['all_modules'])}")
print(f"  • Total methods: {total_methods + len(api_documentation['client_methods'])}")
print(f"  • Total properties: {total_properties}")

print("\n\nDetailed Module Breakdown:")
for module_path, module_info in sorted(api_documentation['all_modules'].items()):
    print(f"\n  {module_path.upper()}")
    print(f"  {'-' * len(module_path)}")
    print(f"  Path: {module_info['path']}")
    print(f"  Type: {module_info['type']}")
    print(f"  Methods: {len(module_info['methods'])}")
    
    if module_info['methods']:
        print(f"  Available operations:")
        for method in sorted(module_info['methods'], key=lambda x: x['name'])[:10]:  # Show first 10
            print(f"    • {method['name']}{method['signature']}")
        if len(module_info['methods']) > 10:
            print(f"    ... and {len(module_info['methods']) - 10} more")
    
    if module_info.get('properties'):
        print(f"  Properties: {', '.join(p['name'] for p in module_info['properties'][:5])}")
        if len(module_info['properties']) > 5:
            print(f"    ... and {len(module_info['properties']) - 5} more")